## Test the model in Images

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath = os.path.sep.join([r'C:\Users\Nirmalya Majhi\Desktop\Fash Mask Detection\Face Detector','deploy.prototxt'])
weightsPath = os.path.sep.join([r'C:\Users\Nirmalya Majhi\Desktop\Fash Mask Detection\Face Detector','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
net = cv2.dnn.readNet(prototxtPath,weightsPath)

#### Load the the saved model from the local computer

In [4]:
model = load_model(r'C:\Users\Nirmalya Majhi\Desktop\Fash Mask Detection\mobilenet_v2_model.h5')

In [5]:
image = cv2.imread(r'C:\Users\Nirmalya Majhi\Desktop\Fash Mask Detection\examples\example1.png')

In [6]:
image

array([[[ 29,  22,  23],
        [ 29,  22,  23],
        [ 29,  22,  23],
        ...,
        [ 30,  21,  22],
        [ 30,  21,  22],
        [ 30,  21,  22]],

       [[ 29,  22,  23],
        [ 29,  22,  23],
        [ 29,  22,  23],
        ...,
        [ 30,  21,  22],
        [ 30,  21,  22],
        [ 30,  21,  22]],

       [[ 29,  22,  23],
        [ 29,  22,  23],
        [ 29,  22,  23],
        ...,
        [ 30,  21,  22],
        [ 30,  21,  22],
        [ 30,  21,  22]],

       ...,

       [[161, 140, 135],
        [226, 205, 200],
        [247, 226, 221],
        ...,
        [253, 247, 244],
        [253, 247, 244],
        [253, 247, 244]],

       [[230, 209, 203],
        [240, 219, 214],
        [237, 216, 210],
        ...,
        [253, 247, 244],
        [253, 247, 244],
        [253, 247, 244]],

       [[239, 218, 213],
        [248, 228, 222],
        [247, 226, 221],
        ...,
        [253, 247, 244],
        [253, 247, 244],
        [253, 247, 244]]

In [7]:
(h , w) = image.shape[:2]

In [8]:
blob = cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [9]:
blob

array([[[[ -75.,  -75.,  -75., ...,  -74.,  -74.,  -74.],
         [ -75.,  -75.,  -75., ...,  -74.,  -74.,  -74.],
         [ -75.,  -75.,  -74., ...,  -74.,  -74.,  -74.],
         ...,
         [  -7.,   91.,  137., ...,  149.,  149.,  149.],
         [  98.,  133.,  140., ...,  149.,  149.,  149.],
         [ 135.,  142.,  137., ...,  149.,  149.,  149.]],

        [[-155., -155., -155., ..., -156., -156., -156.],
         [-155., -155., -155., ..., -156., -156., -156.],
         [-155., -155., -155., ..., -156., -156., -156.],
         ...,
         [ -98.,   -1.,   45., ...,   70.,   70.,   70.],
         [   4.,   39.,   46., ...,   70.,   70.,   70.],
         [  41.,   48.,   43., ...,   70.,   70.,   70.]],

        [[-100., -100., -100., ..., -101., -101., -101.],
         [-100., -100., -100., ..., -101., -101., -101.],
         [-100., -100., -100., ..., -101., -101., -101.],
         ...,
         [ -50.,   47.,   94., ...,  121.,  121.,  121.],
         [  52.,   88.,   

In [10]:
blob.shape

(1, 3, 300, 300)

In [11]:
net.setInput(blob)
detections = net.forward()

#### loop over the detections

In [13]:
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    
    if confidence > 0.5 :
        #we need the X,Y coordinates
        box = detections[0,0,i,3:7] * np.array([w,h,w,h])
        (startX,startY,endX,endY) = box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the framemax
        (startX,startY) = (max(0,startX), max(0,startY))
        (endX,endY) = (min(w - 1,endX), min(h - 1,endY))
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face = cv2.resize(face,(224,224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face,axis=0)
        
        (mask,withoutMask) = model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label = 'Mask' if mask > withoutMask else 'No Mask'
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        
        #include the probability in the label
        label = "{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
cv2.imshow("Output",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 1s 635ms/step
